In [1]:
import pandas as pd

file_path = '../data/melb_data.csv'

home_data = pd.read_csv(file_path)
home_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [2]:
y = home_data.Price

feature_names = ['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']

# Entferne Features mit zu vielen fehlenden Werten
available_features = []
for feature in feature_names:
    if feature in home_data.columns:
        available_features.append(feature)
        print(f"{feature}: {home_data[feature].isnull().sum()} fehlende Werte")

# Verwende nur Features, die tatsächlich existieren
X = home_data[available_features].dropna()
y = y[X.index]  # Sorge dafür, dass y die gleichen Indizes hat wie X

print(f"\nVerwende Features: {available_features}")
print(f"Datensatz-Größe nach Entfernen fehlender Werte: {len(X)}")


Rooms: 0 fehlende Werte
Distance: 0 fehlende Werte
Bedroom2: 0 fehlende Werte
Bathroom: 0 fehlende Werte
Car: 62 fehlende Werte
Landsize: 0 fehlende Werte
BuildingArea: 6450 fehlende Werte
YearBuilt: 5375 fehlende Werte

Verwende Features: ['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']
Datensatz-Größe nach Entfernen fehlender Werte: 6830


In [3]:
# Review data
# print description or statistics from X
print("Statistiken der Features:")
print(X.describe())
#pd.DataFrame.describe(home_data)

Statistiken der Features:
             Rooms     Distance     Bedroom2     Bathroom          Car  \
count  6830.000000  6830.000000  6830.000000  6830.000000  6830.000000   
mean      2.978184    10.148960     2.951391     1.594143     1.606881   
std       0.970479     5.991423     0.970789     0.714366     0.944613   
min       1.000000     0.000000     0.000000     1.000000     0.000000   
25%       2.000000     6.100000     2.000000     1.000000     1.000000   
50%       3.000000     9.200000     3.000000     1.000000     2.000000   
75%       4.000000    13.000000     4.000000     2.000000     2.000000   
max       8.000000    47.400000     9.000000     8.000000    10.000000   

           Landsize  BuildingArea    YearBuilt  
count   6830.000000   6830.000000  6830.000000  
mean     487.495461    143.446606  1964.444070  
std      910.805627     89.970692    37.706332  
min        0.000000      0.000000  1196.000000  
25%      167.000000     93.000000  1940.000000  
50%      404.

In [4]:
from sklearn.tree import DecisionTreeRegressor
#specify the model.
#For model reproducibility, set a numeric value for random_state when specifying the model
iowa_model = DecisionTreeRegressor(random_state=1)

# Fit the model
iowa_model.fit(X, y)

print("Modell erfolgreich trainiert!")

# Zeige ein paar Vorhersagen
predictions = iowa_model.predict(X.head())
print(f"Erste paar Vorhersagen: {predictions}")
print(f"Tatsächliche Werte: {y.head().values}")

Modell erfolgreich trainiert!
Erste paar Vorhersagen: [1035000. 1465000. 1600000. 1876000. 1636000.]
Tatsächliche Werte: [1035000. 1465000. 1600000. 1876000. 1636000.]


In [5]:
# You can write code in this cell
pd.DataFrame(predictions).head()

,0
0,1035000.0
1,1465000.0
2,1600000.0
3,1876000.0
4,1636000.0


In [6]:
def schaetze_hauspreis(rooms, distance, bedroom2, bathroom, car, landsize, building_area, year_built):
    """
    Schätzt den Hauspreis basierend auf den eingegebenen Eigenschaften
    """
    # Erstelle einen DataFrame mit den eingegebenen Werten
    haus_eigenschaften = pd.DataFrame({
        'Rooms': [rooms],
        'Distance': [distance],
        'Bedroom2': [bedroom2],
        'Bathroom': [bathroom],
        'Car': [car],
        'Landsize': [landsize],
        'BuildingArea': [building_area],
        'YearBuilt': [year_built]
    })

    # Vorhersage mit dem trainierten Modell
    geschaetzter_preis = iowa_model.predict(haus_eigenschaften)[0]

    print(f"Geschätzter Hauspreis: ${geschaetzter_preis:,.2f}")

    # Zeige die eingegebenen Eigenschaften
    print("\nEingegebene Eigenschaften:")
    for feature, value in haus_eigenschaften.iloc[0].items():
        print(f"  {feature}: {value}")

    return geschaetzter_preis

In [7]:
# Beispiel: Schätze den Preis für ein eigenes Haus
# Passen Sie diese Werte an Ihr Haus an:

mein_haus_preis = schaetze_hauspreis(
    rooms=3,  # Anzahl Zimmer
    distance=10.5,  # Entfernung zum Stadtzentrum (km)
    bedroom2=3,  # Anzahl Schlafzimmer
    bathroom=2,  # Anzahl Badezimmer
    car=2,  # Anzahl Garagenplätze
    landsize=600,  # Grundstücksgröße (m²)
    building_area=150,  # Gebäudefläche (m²)
    year_built=2000  # Baujahr
)

Geschätzter Hauspreis: $750,000.00

Eingegebene Eigenschaften:
  Rooms: 3.0
  Distance: 10.5
  Bedroom2: 3.0
  Bathroom: 2.0
  Car: 2.0
  Landsize: 600.0
  BuildingArea: 150.0
  YearBuilt: 2000.0
